# Removing Clouds and Clouds Shadows from Landsat8 imagery
**Problem:** Some regions are covered mostly through clouds such that selecting the least cloudy image isn't enough.

In [108]:
import ee
import geemap

EE_PROJECT_NAME = 'seeing-the-big-picture'

try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise e

# Region in Equador close to training sample airport 32 - tropical climate, mostly cloudy 
bbox = [-78.28694022837181, -1.8140575646754744,
        -78.4420222533541, -1.6691395896577617]

center_coords = [-1.73, -78.35]

region = ee.Geometry.Rectangle(bbox)

optical_bands = ['SR_B4', 'SR_B3', 'SR_B2']
vis_params = {'bands': optical_bands, 'min': 0, 'max': 0.3}

def scale_l8(image):
    scaled_optical_bands = (image
            .select(optical_bands)
            .multiply(0.0000275)
            .add(-0.2))
    return image.addBands(scaled_optical_bands, optical_bands, True) 

l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
       .filterBounds(region)
       .map(scale_l8))
       
m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    l8.sort('CLOUD_COVER').first(),
    vis_params,
    'Least Cloudy by Scene'
)
m.addLayer(region, None, 'Region')
m.addLayerControl()
m

Map(center=[-1.73, -78.35], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

To overcome this problem, cloudy pixels can be masked away in a collection and create a composite from the cloud free collection:
1. Mask cloudy pixels.
2. Check if a single image is (almost) cloud free.
3. If not, create a composite.

In [106]:
def mask_l8_clouds(image: ee.Image) -> ee.Image:
    """Updates an image mask, to filter out cloudy pixels.
    
    For a detailed description of the 'QA_PIXEL' flags see:
        https://www.usgs.gov/landsat-missions/landsat-collection-2-quality-assessment-bands

    Args:
        image (ee.Image) 

    Returns:
        ee.Image: Image with updated mask.
    """
    qa = image.select('QA_PIXEL')   
    # Only pixels for which the first 5 bits equal zero are not masked away.
    cloud_mask = qa.bitwiseAnd(int('11111', 2)).eq(0) 
    return image.updateMask(cloud_mask)

def compute_validity_fraction(image: ee.Image, lazy: bool=True) -> ee.Number:
    """Computes the fraction of valid pixels of an image.
    
    Valid pixels are those, which are not masked away, i.e. whose 
    mask value is equal to one.

    Args:
        image (ee.Image): Image to compute the validity for. 
        lazy (bool): Specifies wheter the API call should be stacked or executed directly.
    Returns:
        ee.Number: Fraction of pixels, which are not masked. 
    """
    validity = ee.Number(
        image.select(optical_bands)
      .mask()
      .reduce(ee.Reducer.min())
      .reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=30,
            maxPixels=1e7)
      .get('min')
    )
    return validity if lazy else validity.getInfo()

def add_validity(collection: ee.ImageCollection) -> ee.ImageCollection:
    """Stacks API call to compute the fraction of valid pixels for each image in l8."""
    return collection.map(lambda img: img.set('validity', compute_validity_fraction(img)))

def get_least_cloudy_single_image(collection: ee.ImageCollection, region: ee.Geometry) -> ee.Image:
    """Return least cloudy image of the collection for the region."""
    return add_validity(collection).sort('validity', False).first()

l8_cloud_masked = l8.map(mask_l8_clouds)
least_cloudy = get_least_cloudy_single_image(l8_cloud_masked, region) 
print(least_cloudy.get('validity').getInfo())

composite_mosaic = l8_cloud_masked.mosaic()
print(compute_validity_fraction(composite_mosaic, lazy=False))

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    least_cloudy.mask(),
    vis_params,
    'Least Cloudy Single Image Mask'
)
m.addLayer(
    least_cloudy,
    vis_params,
    'Least Cloudy Single Image'
)
m.addLayer(
    composite_mosaic,
    vis_params,
    'Mosaic'
)
m.addLayer(region, None, 'Region')
m.addLayerControl()
m

0.9558998516090057
0.9998319588747859


Map(center=[-1.73, -78.35], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [107]:
def get_least_cloudy(l8: ee.ImageCollection, region: ee.Geometry) -> ee.Image:
    """Return least cloudy image or mosaic composite.
    
    Mosaic composite is returned, if single least cloudy image contains more than 1% of invalid pixels. 
    """
    l8_cloud_masked = l8.map(mask_l8_clouds)
    least_cloudy = get_least_cloudy_single_image(l8_cloud_masked, region) 
    least_cloudy_is_ok = ee.Number(least_cloudy.get('validity')).gte(0.99)
    mosaic = l8_cloud_masked.mosaic()
    return ee.Image(ee.Algorithms.If(least_cloudy_is_ok, least_cloudy, mosaic))

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    get_least_cloudy(l8, region),
    vis_params,
    'Least Cloudy'
)
m.addLayer(region, None, 'Region')
m.addLayerControl()
m

Map(center=[-1.73, -78.35], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…